In [1]:
from sklearn.preprocessing import FunctionTransformer
import pandas as pd
import holidays
import warnings
from merge_transformer import MergeTransformer
from create_db import create_db

warnings.simplefilter(action='ignore', category=FutureWarning)

In [2]:
def check_csv_correct(X):
        
    if X.isna().sum().all() != 0:
        return False
    return True

In [3]:
def merge_dfs(database, submission_dir):
    
    X = database['AirportStatistics']
    
    merge_transform = MergeTransformer(X_ext=database['Date'], how='left', on=['DateOfDeparture'])
    X = merge_transform.fit_transform(X)
    
    merge_transform = MergeTransformer(X_ext=database['Airport'], cols_to_rename={'iata': 'AirPort'}, 
                                       how='inner', on=['AirPort'])
    X = merge_transform.fit_transform(X)
    
    merge_transform = MergeTransformer(X_ext=database['StateFeatures'],
                                       cols_to_keep=['state', 'year', 'month', 'UnemploymentRate', 'holidays', 'GDP_per_cap', 'closest_holidays'],
                                       cols_to_rename={'Abbreviation': 'state'}, how='left', 
                                       on=['year', 'month', 'day', 'state'])
    X = merge_transform.fit_transform(X)
    
    merge_transform = MergeTransformer(X_ext=database['Routes'],
                                       cols_to_rename={'Departure': 'AirPort'},
                                       on=['AirPort'])
    X = merge_transform.fit_transform(X)
    
    all_check = check_csv_correct(X)
    
    if all_check:
        X.to_csv(submission_dir + 'external_data.csv')
    else:
        raise Exception("Sorry will not create CSV as Database has Nans or some other shit. Try Again!")
        
    return all_check

In [4]:
def create_external_data(submission_dir):
    
    database = create_db()
    merge_dfs(database, submission_dir)

In [5]:
create_external_data('../submissions/test/')

<class 'pandas.core.frame.DataFrame'>
Int64Index: 41340 entries, 0 to 41339
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   DateOfDeparture   41340 non-null  datetime64[ns]
 1   year              41340 non-null  int64         
 2   month             41340 non-null  int64         
 3   day               41340 non-null  int64         
 4   weekday           41340 non-null  int64         
 5   week              41340 non-null  int64         
 6   n_days            41340 non-null  int64         
 7   day_nb            41340 non-null  int64         
 8   UnemploymentRate  41340 non-null  float64       
 9   State             41340 non-null  object        
 10  Abbreviation      40545 non-null  object        
 11  GDP_per_cap       40545 non-null  float64       
dtypes: datetime64[ns](1), float64(2), int64(7), object(2)
memory usage: 4.1+ MB
None
